### Import các thư viện cần thiết

In [1]:
# dùng Pytorch để xây dựng và huấn luyện mô hình deep learning
import torch
import torch.nn as nn
import os
import random 
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Sử dụng PIL để xử lý dữ liệu ảnh
from PIL import Image
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split

In [2]:
def set_seed(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

seed = 59
set_seed(seed)

In [3]:
root_dir = './data/weather-dataset/dataset'
img_paths = []
labels = []
classes = {
    label_idx : class_name \
    for label_idx, class_name in enumerate(sorted(os.listdir(root_dir)))
}

In [4]:
for label_idx, class_name in classes.items():
    class_dir = os.path.join(root_dir, class_name)
    for img_filename in os.listdir(class_dir):
        img_path = os.path.join(class_dir, img_filename)
        img_paths.append(img_path)
        labels.append(label_idx)

In [5]:
val_size = 0.2
test_size = 0.125
is_shuffle = True

X_train, X_val, y_train, y_val = train_test_split(
    img_paths, labels,
    test_size=val_size,
    random_state=seed,
    shuffle=is_shuffle,
)

X_train, X_test, y_train, y_test = train_test_split(
    X_train, y_train,
    test_size=val_size,
    random_state=seed,
    shuffle=is_shuffle,
)

In [6]:
class WeatherDataset(Dataset):
    def __init__ (self, X, y, transform = None):
        self.transform = transform
        self.img_paths = X
        self.labels = y

    def __len__(self):
        return len(self.img_paths)
    
    def __getitem__(self, idx):
        img_path = self.img_paths[idx]
        img = Image.open(img_path).convert('RGB')

        if self.transform:
            img = self.transform(img)

        return img, self.labels[idx]

In [7]:
def transform(img, img_size=(224, 224)):
    img = img.resize(img_size)
    img = np.array(img)[..., :3]
    img = torch.tensor(img).permute(2, 0, 1).float()
    normalized_img = img / 255.0

    return normalized_img

In [8]:
train_dataset = WeatherDataset(X_train, y_train, transform=transform)
val_dataset = WeatherDataset(X_val, y_val, transform=transform)
test_dataset = WeatherDataset(X_test, y_test, transform=transform)

In [9]:
train_batch_size = 512
test_batch_size = 8

train_loader = DataLoader(
    train_dataset, 
    batch_size=train_batch_size, 
    shuffle=True
    )

val_loader = DataLoader(
    val_dataset, 
    batch_size=test_batch_size, 
    shuffle=False
    )

test_loader = DataLoader(
    test_dataset, 
    batch_size=test_batch_size, 
    shuffle=False
    )


In [10]:
class ResidualBlock(nn.Module):
    def __init__(self, in_channels, out_channels, stride = 1):
        super(ResidualBlock, self).__init__()
        self.conv1 = nn.Conv2d(in_channels, out_channels, kernel_size=3, stride=stride, padding=1)
        self.bn1 = nn.BatchNorm2d(out_channels)
        self.conv2 = nn.Conv2d(out_channels, out_channels, kernel_size=3, stride=1, padding=1)
        self.bn2 = nn.BatchNorm2d(out_channels)
        self.downsample = nn.Sequential()
        if stride != 1 or in_channels != out_channels:
            self.downsample = nn.Sequential(
                nn.Conv2d(in_channels, out_channels, kernel_size=1, stride=stride),
                nn.BatchNorm2d(out_channels)
            )
        self.relu = nn.ReLU()

    def forward(self, x):
        shortcut = x.clone()
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.conv2(x)
        x = self.bn2(x)
        x += self.downsample(shortcut)
        x = self.relu(x)

        return x

In [11]:
class ResNet(nn.Module):
    def __init__(self, residual_block, n_block_lst, n_classes):
        super(ResNet, self).__init__()
        self.conv1 = nn.Conv2d(3, 64, kernel_size=7, stride=2, padding=3)
        self.bn1 = nn.BatchNorm2d(64)
        self.relu = nn.ReLU()
        self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)
        self.conv2 = self.create_layer(residual_block, 64, 64, n_block_lst[0], 1)
        self.conv3 = self.create_layer(residual_block, 64, 128, n_block_lst[1], 2)
        self.conv4 = self.create_layer(residual_block, 128, 256, n_block_lst[2], 2)
        self.conv5 = self.create_layer(residual_block, 256, 512, n_block_lst[3], 2)
        self.avgpool = nn.AdaptiveAvgPool2d((1))
        self.flatten = nn.Flatten()
        self.fc1 = nn.Linear(512, n_classes)

    def create_layer(self, residual_block, in_channels, out_channels, n_blocks, stride):
        blocks = []
        first_block = residual_block(in_channels, out_channels, stride)
        blocks.append(first_block)

        for idx in range(1, n_blocks):
            block = residual_block(out_channels, out_channels, stride)
            blocks.append(block)

        block_seq = nn.Sequential(*blocks)

        return block_seq
    
    def forward(self, x):
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.maxpool(x)
        x = self.conv2(x)
        x = self.conv3(x)
        x = self.conv4(x)
        x = self.conv5(x)
        x = self.avgpool(x)
        x = self.flatten(x)
        x = self.fc1(x)

        return x

In [12]:
n_classes = len(list(classes.keys()))
device = 'cuda' if torch.cuda.is_available() else 'cpu'

model = ResNet(ResidualBlock, [2, 2, 2, 2], n_classes).to(device)

In [13]:
def evaluate(model, dataloader, criterion, device):
    model.eval()
    correct = 0
    total = 0
    losses = []
    with torch.no_grad():
        for inputs, labels in dataloader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            losses.append(loss.item())
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    loss = sum(losses) / len(losses)
    acc = correct / total

    return loss, acc

In [14]:
def fit(model, train_loader, val_loader, criterion, optimizer, device, epochs):
    train_losses = []
    val_losses = []

    for epoch in range(epochs):
        batch_train_losses = []

        model.train()
        for idx, (inputs, labels) in enumerate(train_loader):
            inputs, labels = inputs.to(device), labels.to(device)

            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            batch_train_losses.append(loss.item())
        
        train_loss = sum(batch_train_losses) / len(batch_train_losses)
        train_losses.append(train_loss)

        val_loss, val_acc = evaluate(model, val_loader, criterion, device)
        val_losses.append(val_loss)

        print(f'Epoch {epoch + 1}/{epochs}, train loss: {train_loss}, val loss: {val_loss}')
    return train_losses, val_losses

In [15]:
lr = 1e-2
epochs = 25

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=lr)
print(device)

cuda


In [16]:
train_losses, val_losses = fit(model, train_loader, val_loader, criterion, optimizer, device, epochs)

Epoch 1/25, train loss: 1.9091659784317017, val loss: 2.3863386497941126, val acc: 0.18135469774217042
Epoch 2/25, train loss: 1.3054147164026897, val loss: 2.3946838947229607, val acc: 0.11507647487254188
Epoch 3/25, train loss: 1.0354009403122797, val loss: 2.356418469617533, val acc: 0.14202476329206118
Epoch 4/25, train loss: 0.8189971711900499, val loss: 2.0903421529503756, val acc: 0.3029861616897305
Epoch 5/25, train loss: 0.6651414102978177, val loss: 1.3792547273427942, val acc: 0.5600873998543335
Epoch 6/25, train loss: 0.5412250823444791, val loss: 1.1966868493446083, val acc: 0.6001456664238893
Epoch 7/25, train loss: 0.4136869079536862, val loss: 1.1527143536612046, val acc: 0.616897305171158
Epoch 8/25, train loss: 0.3254849314689636, val loss: 1.033951195000216, val acc: 0.6613255644573925
Epoch 9/25, train loss: 0.24365728927983177, val loss: 1.1323745501942413, val acc: 0.6190823015294975
Epoch 10/25, train loss: 0.1679245481888453, val loss: 1.0274927874290667, val ac

In [17]:
val_loss, val_acc = evaluate(
    model,
    val_loader,
    criterion,
    device
)

test_loss, test_acc = evaluate(
    model,
    test_loader,
    criterion,
    device
)

print('Evaluation on val/test dataset')
print('Val accuracy: ', val_acc)
print('Test accuracy: ', test_acc)

Evaluation on val/test dataset
Val accuracy:  0.6831755280407866
Test accuracy:  0.692167577413479
